# Airqdata: analysis tools for air quality data

Notebook version with output and functional links: [https://nbviewer.jupyter.org/gist/dr-1/450c275b1ad2cbf88e9c4325c5d032bc](https://nbviewer.jupyter.org/gist/dr-1/450c275b1ad2cbf88e9c4325c5d032bc)

**Table of contents**  
[Civic Labs resources](#Civic-Labs-resources)  
[Madavi.de resources](#Madavi.de-resources)  
[Luftdaten.info resources](#Luftdaten.info-resources)  
[Irceline.be resources](#Irceline.be-resources)  
[Combining the sources](#Combining-the-sources)

In [ ]:
%matplotlib inline

In [ ]:
%run airqdata
pd.set_option("display.max_rows", 10)

## Civic Labs resources

### Download list of sensors from Civic Labs' Google Sheet

In [ ]:
sensor_info = civiclabs.get_sensor_info(refresh_cache=True)
sensor_info.head(4)

In [ ]:
len(sensor_info)

In [ ]:
demo_chip_id = "956245"
demo_sensor_id = "5561"

In [ ]:
sensor_info[sensor_info["PM Sensor ID"] == demo_sensor_id]

## Madavi.de resources

### Open web pages in browser showing data history graphs of a sensor

In [ ]:
# Particulate matter
madavi.open_graphs(demo_chip_id)

# Temperature and humidity
madavi.open_graphs(demo_chip_id, sensor_model="dht")

## Luftdaten.info resources

### Create a Sensor object and get the sensor's current data

In [ ]:
demo_sensor = luftdaten.Sensor(demo_sensor_id, refresh_cache=True)

In [ ]:
demo_sensor.metadata

In [ ]:
demo_sensor.metadata_url

In [ ]:
demo_sensor.current_measurements

### Retrieve data history
Data are retrieved from cache or server and then cleaned.

In [ ]:
demo_sensor.get_measurements(start_date="2018-02-04", end_date="2018-02-08")

### Inspect, summarize and plot data

In [ ]:
demo_sensor.measurements

In [ ]:
describe(demo_sensor.measurements)

In [ ]:
demo_sensor.plot_measurements()

### Inspect, summarize and plot hourly means

In [ ]:
demo_sensor.get_hourly_means()

In [ ]:
describe(demo_sensor.get_hourly_means())

In [ ]:
demo_sensor.plot_hourly_means()

### Check distribution of sample intervals
In seconds

In [ ]:
demo_sensor.intervals.head(10)

### List sensors near a given location

Defaults to searching within an 8 kilometer radius around the center of Brussels

In [ ]:
near = luftdaten.search_proximity()
near

Sensors near Antwerp

In [ ]:
luftdaten.search_proximity(lat=51.22, lon=4.41, radius=20)

In [ ]:
(near_sensors,
 hourly_means) = luftdaten.evaluate_near_sensors(start_date="2017-09-10",
                                                 end_date="2017-09-13",
                                                 quiet=True)

In [ ]:
hourly_means

## Irceline.be resources

### Get IRCELINE metadata about phenomena and stations

In [ ]:
irceline.Metadata()

In [ ]:
irceline.Metadata.phenomena

In [ ]:
irceline.Metadata.stations

In [ ]:
irceline.Metadata.get_stations_by_name("bru")

In [ ]:
irceline.Metadata.time_series

### How many stations measure a given phenomenon?

In [ ]:
irceline.Metadata.time_series["phenomenon"].value_counts()

### How many phenomena does a given station measure?

In [ ]:
irceline.Metadata.time_series["station_label"].value_counts().head()

In [ ]:
pd.set_option("display.max_rows", 6)

### Where is a given phenomenon measured?

In [ ]:
irceline.Metadata.query_time_series(phenomenon="ethylbenzene")

### Where is PM2.5 measured?

In [ ]:
irceline.Metadata.get_pm25_time_series()

### Where is PM10 measured?

In [ ]:
irceline.Metadata.get_pm10_time_series()

### What are the closest locations to Etterbeek where IRCELINE measures NO₂?
Using a location in Etterbeek as a reference point: 50.837°N 4.39°E

In [ ]:
irceline.Metadata.query_time_series("nitrogen dioxide", lat_nearest=50.837, lon_nearest=4.39)

### What does the Uccle station measure?

In [ ]:
irceline.Metadata.list_station_time_series("ucc")

### List stations near a location
Defaults to coordinates and radius of Brussels

In [ ]:
irceline.Metadata.search_proximity(lat=50.9, lon=4.4, radius=5)

### Create a sensor object from a time series, retrieve its measurements and plot them

In [ ]:
irceline_demo_sensor = irceline.Sensor("6615")  # An NO₂ sensor in Ixelles

In [ ]:
irceline_demo_sensor.get_measurements(start_date="2018-02-03", end_date="2018-02-08")

In [ ]:
irceline_demo_sensor.measurements.head()

In [ ]:
irceline_demo_sensor.plot_measurements()

## Combining the sources

In [ ]:
pd.set_option("display.max_rows", 10)

### Which are the closest IRCELINE sensors to a given luftdaten.info sensor that measure the same phenomenon?

In [ ]:
nearest = irceline.find_nearest_sensors(demo_sensor, quiet=True)
nearest

### Compare data of a luftdaten.info sensor and the nearest IRCELINE sensors

In [ ]:
combined_data, plots = compare_nearest_irceline_sensors(demo_sensor,
                                                        start_date="2018-02-03", end_date="2018-02-10",
                                                        quiet=True)

#### Correlation between the compared values

In [ ]:
combined_data.corr()

### Compare data from any sensors

In [ ]:
t_rh_sensor = luftdaten.Sensor("5562")  # Temperature and humidity sensor at Brussels Central Station
combined_data, plot = compare_sensor_data(sensors=[demo_sensor, t_rh_sensor, t_rh_sensor, irceline_demo_sensor],
                                          phenomena=["pm2.5", "temperature", "humidity", "Nitrogen dioxide"],
                                          start_date="2018-02-05", end_date="2018-02-10",
                                          hourly_means=True,
                                          quiet=True)

In [ ]:
combined_data.head()

#### Correlation between the compared values

In [ ]:
combined_data.corr()

## Export data for use in another environment

In [ ]:
# demo_sensor.measurements.to_csv("demo_sensor_data.csv")